Imports all required libraries:

numpy, pandas: for numerical and tabular data processing

plotly: for interactive 3D visualizations

stable_baselines3: to use PPO (Proximal Policy Optimization) algorithm

gymnasium: to define and manage custom reinforcement learning environment

In [16]:
!pip install stable-baselines3[extra] gym numpy plotly
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from stable_baselines3 import PPO
import gymnasium as gym
from gymnasium import spaces


The `DroneEnv` class defines a custom 3D drone environment using OpenAI Gym. It simulates a drone navigating from a start to a goal position within a limited time, with movement in 3D space and time as part of the state. The drone earns rewards for moving closer to the goal and gets a high reward upon reaching it. The environment tracks the drone's trajectory and ends either when the goal is reached or time runs out.


In [17]:
class DroneEnv(gym.Env):
    def __init__(self, start, goal, time_limit=80):
        super().__init__()
        self.start = np.array(start, dtype=np.float32)
        self.goal = np.array(goal, dtype=np.float32)
        self.time_limit = time_limit
        self.space_size = 20
        self.threshold = 4.0
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0], dtype=np.float32),
            high=np.array([self.space_size, self.space_size, self.space_size, self.time_limit], dtype=np.float32),
            dtype=np.float32
        )
        self.action_space = spaces.Box(
            low=np.array([-1, -1, -1, 1], dtype=np.float32),
            high=np.array([1, 1, 1, 5], dtype=np.float32),
            dtype=np.float32
        )
        self.reset()

    def reset(self, seed=None, options=None):
        self.pos = self.start.copy()
        self.time = 0.0
        self.trajectory = []
        self.location_log = []
        return self._get_obs(), {}

    def _get_obs(self):
        return np.array([*self.pos, self.time], dtype=np.float32)

    def step(self, action):
        self.pos += action[:3]
        self.pos = np.clip(self.pos, 0, self.space_size)
        self.time += action[3]
        self.time = min(self.time, self.time_limit)

        distance_to_goal = np.linalg.norm(self.pos - self.goal)
        reward = -0.5
        terminated = False
        truncated = False

        if distance_to_goal < 2:
            reward = 200
            terminated = True
        elif self.time >= self.time_limit:
            truncated = True
        else:
            reward += max(0, 5 - distance_to_goal) * 0.1

        self.trajectory.append((self.pos.copy(), self.time))
        self.location_log.append((self.time, *self.pos))
        return self._get_obs(), reward, terminated, truncated, {}

This block initializes a multi-agent drone simulation by defining six drones, each with specified starting positions, goal destinations, and unique colors for visualization. It prepares the environment for training by setting up an empty list (`trained_paths`) to store the trained trajectories of each drone and an empty DataFrame (`locations_df`) to log their positions over time. The `time_limit` is set to 80, defining how long a drone can operate per episode, and `episodes_per_agent` is set to 3, indicating that each drone will undergo three training episodes using reinforcement learning.


In [18]:
agents = [
    {'start': [0, 0, 0], 'goal': [15, 15, 10], 'color': 'blue'},
    {'start': [0, 15, 0], 'goal': [15, 0, 10], 'color': 'green'},
    {'start': [10, 0, 0], 'goal': [0, 15, 10], 'color': 'orange'},
    {'start': [0, 10, 0], 'goal': [15, 5, 12], 'color': 'purple'},
    {'start': [5, 0, 5], 'goal': [15, 15, 15], 'color': 'brown'},
    {'start': [0, 5, 2], 'goal': [18, 8, 10], 'color': 'cyan'}
]

trained_paths = []
locations_df = pd.DataFrame()
time_limit = 80
episodes_per_agent = 3

This block trains each drone agent using Proximal Policy Optimization (PPO) for a defined number of episodes (3 in this case). For each episode, it creates a new drone environment with the agent’s start and goal positions, trains the PPO model for 8000 timesteps, and evaluates the agent’s performance. It keeps track of the best-performing episode based on the total reward, storing the best model, trajectory, and location log. After training, the best trajectory is saved, and the location data (with time and drone identity) is added to a cumulative DataFrame (locations_df) for visualization and further analysis

In [19]:
for i, agent in enumerate(agents):
    best_model = None
    best_reward = float('-inf')
    best_trajectory = None
    best_location_log = None

    for ep in range(episodes_per_agent):
        env = DroneEnv(agent['start'], agent['goal'], time_limit)
        model = PPO("MlpPolicy", env, verbose=0)
        model.learn(total_timesteps=8000)

        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = model.predict(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated

        if total_reward > best_reward:
            best_model = model
            best_reward = total_reward
            best_trajectory = env.trajectory.copy()
            best_location_log = env.location_log.copy()

    agent['trajectory'] = [(p[0], p[1]) for p in best_trajectory]
    trained_paths.append(best_trajectory)

    df_loc = pd.DataFrame(best_location_log, columns=['time', 'x', 'y', 'z'])
    df_loc['drone'] = f"Drone {i+1}"
    locations_df = pd.concat([locations_df, df_loc], ignore_index=True)


This code creates 3D line plots of each drone's flight path using Plotly. It maps each drone to its color, filters its position data from `locations_df`, and generates a dotted `Scatter3d` line showing its movement in space. These lines are saved in `trace_lines` for use in the final visualization.



In [20]:
drone_colors = {f"Drone {i+1}": agents[i]['color'] for i in range(len(agents))}

trace_lines = []
for drone in locations_df["drone"].unique():
    df_path = locations_df[locations_df["drone"] == drone]
    trace_lines.append(go.Scatter3d(
        x=df_path["x"], y=df_path["y"], z=df_path["z"],
        mode="lines",
        line=dict(dash="dot", width=2, color=drone_colors[drone]),
        name=f"{drone} Path"
    ))


This code creates a 3D animation of drone movements over time using Plotly. It builds time-based frames showing each drone's position with hover details and combines them with static path lines. The layout sets up the 3D scene and adds a play button to animate drone paths in real-time.


In [21]:
frames = []
for t in range(time_limit + 1):
    df_t = locations_df[locations_df["time"] == t]
    frame_data = []
    for drone in df_t["drone"].unique():
        row = df_t[df_t["drone"] == drone].iloc[0]
        hover_text = f"{drone}<br>Time: {int(row['time'])}<br>X: {row['x']:.2f}<br>Y: {row['y']:.2f}<br>Z: {row['z']:.2f}"
        frame_data.append(go.Scatter3d(
            x=[row["x"]], y=[row["y"]], z=[row["z"]],
            mode="markers+text",
            marker=dict(size=6, color=drone_colors[drone]),
            text=[hover_text],
            name=drone,
            hoverinfo='text'
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

layout = go.Layout(
    title="🛰️ 3D UAV Deconfliction Animation with Paths",
    scene=dict(
        xaxis=dict(title='X', range=[0, 20]),
        yaxis=dict(title='Y', range=[0, 20]),
        zaxis=dict(title='Z', range=[0, 20])
    ),
    width=950,
    height=700,
    updatemenus=[dict(type='buttons', showactive=False,
                      buttons=[dict(label='▶️ Play', method='animate',
                                    args=[None, {
                                        "frame": {"duration": 300, "redraw": True},
                                        "fromcurrent": True,
                                        "mode": "immediate"
                                    }])])]
)

fig_3d = go.Figure(data=trace_lines + list(frames[0].data), layout=layout, frames=frames)
fig_3d.show()

 A multi-line subplot visualization showing how each drone's position changes over time along the X, Y, and Z axes is created. Using `make_subplots`, it arranges three vertically stacked plots (one for each axis). For every drone in the `locations_df`, it plots a line with markers for X, Y, and Z positions against time.

In [22]:
fig_pos = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        subplot_titles=["Time vs X Position", "Time vs Y Position", "Time vs Z Position"])

for drone in locations_df["drone"].unique():
    df_d = locations_df[locations_df["drone"] == drone]
    fig_pos.add_trace(go.Scatter(x=df_d["time"], y=df_d["x"], mode='lines+markers', name=f'{drone} - X'), row=1, col=1)
    fig_pos.add_trace(go.Scatter(x=df_d["time"], y=df_d["y"], mode='lines+markers', name=f'{drone} - Y'), row=2, col=1)
    fig_pos.add_trace(go.Scatter(x=df_d["time"], y=df_d["z"], mode='lines+markers', name=f'{drone} - Z'), row=3, col=1)

fig_pos.update_layout(height=900, width=1000, title_text="📈 Drone Position Over Time")
fig_pos.update_xaxes(title_text="Time", row=3, col=1)
fig_pos.update_yaxes(title_text="X Position", row=1, col=1)
fig_pos.update_yaxes(title_text="Y Position", row=2, col=1)
fig_pos.update_yaxes(title_text="Z Position", row=3, col=1)

fig_pos.show()

In [23]:
locations_df.to_csv("drone_position_log.csv", index=False)
print("Drone position log saved to 'drone_position_log.csv'")
print("Sample Drone Position Log:")
print(locations_df.head(10).to_string(index=False))

Drone position log saved to 'drone_position_log.csv'
Sample Drone Position Log:
     time        x        y        z   drone
 1.000000 0.000000 0.193996 1.000000 Drone 1
 2.000000 0.149529 0.000000 0.000000 Drone 1
 3.000000 0.073662 0.123594 0.000000 Drone 1
 4.000000 1.073662 1.123594 0.000000 Drone 1
 5.000000 0.073662 0.443996 0.253083 Drone 1
 6.106676 0.845175 1.246101 1.253083 Drone 1
 7.106676 1.715177 1.101620 0.253083 Drone 1
 8.106676 2.474054 0.101620 0.000000 Drone 1
 9.106676 1.828592 0.000000 1.000000 Drone 1
10.106676 1.711170 0.298006 0.765356 Drone 1


This code reads a CSV file containing drone position data over time and visualizes sudden position changes ("spikes") in a line chart using Plotly. It calculates the 3D position magnitude at each time step and then computes the difference between consecutive magnitudes to identify movement changes. These differences are plotted over time as red spikes, helping detect anomalies or rapid movements in the drone's path.


In [24]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv("drone_position_log.csv")

df["position_magnitude"] = (df["x"]**2 + df["y"]**2 + df["z"]**2)**0.5
df["position_change"] = df["position_magnitude"].diff()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["time"],
    y=df["position_change"],
    mode="lines+markers",
    name="Position Change (Spike)",
    line=dict(color="red"),
))

fig.update_layout(
    title="Drone Position Spike Graph",
    xaxis_title="Time (s)",
    yaxis_title="Change in Position Magnitude",
    template="plotly_dark"
)

fig.show()


This code generates simulated 3D positions for 4 drones over time using random initial positions and velocities, with sinusoidal variations added for realism. The data is stored in a pandas DataFrame for visualization or analysis.


In [25]:
import plotly.graph_objects as go
num_drones = 4
time_limit = 30
np.random.seed(42)
data = []
for drone_id in range(num_drones):
    x, y, z = np.random.uniform(1, 5), np.random.uniform(1, 5), np.random.uniform(1, 5)
    vx, vy, vz = np.random.uniform(0.2, 0.6), np.random.uniform(0.2, 0.6), np.random.uniform(0.2, 0.6)
    for t in range(time_limit + 1):
        x_t = x + vx * t + np.sin(t / 3 + drone_id)
        y_t = y + vy * t + np.cos(t / 3 + drone_id)
        z_t = z + vz * t + np.sin(t / 5 + drone_id)
        data.append({"drone": f"drone_{drone_id}", "time": t, "x": x_t, "y": y_t, "z": z_t})
df = pd.DataFrame(data)

This segment creates 3D trajectory plots for each drone by assigning distinct colors and mapping their movement through space. For every drone, it extracts position data from the dataset and generates a line trace that represents its flight path across the X, Y, and Z axes.

In [26]:
drone_colors = {f"drone_{i}": c for i, c in enumerate(["red", "blue", "green", "orange", "purple", "cyan"])}
trace_lines = []
for drone in df["drone"].unique():
    df_d = df[df["drone"] == drone]
    trace_lines.append(go.Scatter3d(
        x=df_d["x"], y=df_d["y"], z=df_d["z"],
        mode='lines',
        name=f"{drone} Path",
        line=dict(color=drone_colors[drone], width=3)
    ))

This part builds animation frames to visualize drone movement over time. For each time step, it filters the dataset to get the position of each drone and creates a 3D marker with hover text showing the drone's ID, coordinates, and time.

In [27]:
frames = []
for t in range(time_limit + 1):
    df_t = df[df["time"] == t]
    frame_data = []
    for drone in df_t["drone"].unique():
        row = df_t[df_t["drone"] == drone].iloc[0]
        hover_text = f"{drone}<br>Time: {t}<br>X: {row['x']:.2f}<br>Y: {row['y']:.2f}<br>Z: {row['z']:.2f}"
        frame_data.append(go.Scatter3d(
            x=[row["x"]], y=[row["y"]], z=[row["z"]],
            mode="markers+text",
            marker=dict(size=6, color=drone_colors[drone]),
            text=[hover_text],
            name=drone,
            hoverinfo='text',
            showlegend=False
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

This sets up a 3D plot with axis labels, title, and play button. It combines drone paths and animated frames to visualize drone movement over time.


In [28]:
layout = go.Layout(
    title="🛰️ Simulated Drone Flight (No Obstacles)",
    scene=dict(
        xaxis=dict(title='X', range=[0, 50]),
        yaxis=dict(title='Y', range=[0, 50]),
        zaxis=dict(title='Z', range=[0, 50])
    ),
    width=950,
    height=700,
    updatemenus=[dict(type='buttons', showactive=False,
                      buttons=[dict(label='▶️ Play', method='animate',
                                    args=[None, {
                                        "frame": {"duration": 300, "redraw": True},
                                        "fromcurrent": True,
                                        "mode": "immediate"
                                    }])])]
)

fig = go.Figure(data=trace_lines + list(frames[0].data), layout=layout, frames=frames)
fig.show()

In [29]:
df.to_csv("simulated_drone_position_log.csv", index=False)
print("Drone simulation saved to 'simulated_drone_position_log.csv'")

Drone simulation saved to 'simulated_drone_position_log.csv'


This reads simulated drone position data from a CSV, calculates the magnitude and change in position over time, and plots a spike graph using Plotly to visualize sudden movement or changes in drone trajectory.

In [30]:
import pandas as pd
import plotly.graph_objects as go

df1 = pd.read_csv("/content/simulated_drone_position_log.csv")

df1["position_magnitude"] = (df["x"]**2 + df["y"]**2 + df["z"]**2)**0.5
df1["position_change"] = df1["position_magnitude"].diff()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1["time"],
    y=df1["position_change"],
    mode="lines+markers",
    name="Position Change (Spike)",
    line=dict(color="red"),
))

fig.update_layout(
    title="Drone Position Spike Graph",
    xaxis_title="Time (s)",
    yaxis_title="Change in Position Magnitude",
    template="plotly_dark"
)

fig.show()
